In [ ]:
import tensorflow as tf

In [ ]:
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
import time
import re
tokenizer = nltk.RegexpTokenizer(r'''\w'|\w+|[^\w\s]''')
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense,Dropout,Embedding,LSTM
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
random.seed(123)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
df_train = pd.read_csv("A.csv")
df_dev = pd.read_csv("A.csv")
df_test = pd.read_csv("A.csv")

In [ ]:
# meta data integration

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
ids = df_train['user_id'].values.tolist() + df_dev['user_id'].values.tolist() + df_test['user_id'].values.tolist()

In [ ]:
len(ids)

In [ ]:
le.fit(ids)

In [ ]:
from textblob import Blobber

In [ ]:
from textblob_fr import PatternTagger, PatternAnalyzer

In [ ]:
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

In [ ]:
polarities = []
for i in range(len(df_train)):
    blob = tb(df_train['commentaire'][i])
    polarities.append(blob.sentiment[0])
    
df_train['polarity'] = polarities
df_train['u_Id']= le.transform(df_train['user_id'].values.tolist())

In [ ]:
polarities_dev = []
for i in range(len(df_dev)):
    blob = tb(df_dev['commentaire'][i])
    polarities_dev.append(blob.sentiment[0])

df_dev['polarity'] = polarities_dev
df_dev['u_Id']= le.transform(df_dev['user_id'].values.tolist())

In [ ]:
polarities_test = []
for i in range(len(df_test)):
    blob = tb(df_test['commentaire'][i])
    polarities_test.append(blob.sentiment[0])

df_test['polarity'] = polarities_test
df_test['u_Id']= le.transform(df_test['user_id'].values.tolist())

In [ ]:
X_train = df_train['commentaire']

In [ ]:
X_dev = df_dev['commentaire']
X_test  = df_test['commentaire']

In [ ]:
y_train = df_train['note']
y_dev = df_dev['note']

In [ ]:
tokenizer = Tokenizer(num_words=len(list(X_train.unique())))

In [ ]:
tokenizer.fit_on_texts(list(X_train))

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train) 
X_dev = tokenizer.texts_to_sequences(X_dev)

In [ ]:
# l = [len(item) for item in X_train ]
# sns.distplot(l, hist=True, kde=True, 
#              bins=int(50), color = 'darkblue', 
#              hist_kws={'edgecolor':'black'},
#              kde_kws={'linewidth': 4})

In [ ]:
# sns.histplot(x=l)

In [ ]:
len_max = 180

In [ ]:
X_train = pad_sequences(X_train, maxlen=len_max)
X_dev = pad_sequences(X_dev, maxlen=len_max)

In [ ]:

unique_words = set()


for sent in tqdm(X_train):
    
    unique_words.update(sent)
    
#     if(len_max<len(sent)):
#         len_max = len(sent)
        
print(len(list(unique_words)))
print(len_max)

In [ ]:
num_classes = len(y_train.unique())
num_classes

In [ ]:
# X_train[9]

In [ ]:
x_train = X_train.tolist()
for i in range(len(X_train)): 
    x_train[i].insert(0,df_train['movie'][i])
    x_train[i].insert(0,df_train['u_Id'][i])
    x_train[i].insert(0,df_train['polarity'][i])
x_train = np.array(x_train)

In [ ]:
# x_train[0]

In [ ]:
x_dev = X_dev.tolist()
for i in range(len(X_dev)): 
    x_dev[i].insert(0,df_dev['movie'][i])
    x_dev[i].insert(0,df_dev['u_Id'][i])
    x_dev[i].insert(0,df_dev['polarity'][i])
x_dev = np.array(x_dev)

In [ ]:
num_classes = 10

In [ ]:
# LSTM_MetaData

In [ ]:
model_mdata=Sequential()
model_mdata.add(Embedding(10000,150,input_length=183))
model_mdata.add(LSTM(128,dropout=0.5, recurrent_dropout=0.5,return_sequences=True))
model_mdata.add(LSTM(256,dropout=0.5, recurrent_dropout=0.5,return_sequences=True))
model_mdata.add(LSTM(64,dropout=0.5, recurrent_dropout=0.5,return_sequences=True))
model_mdata.add(LSTM(100,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model_mdata.add(Dense(64,activation='relu'))
model_mdata.add(Dropout(0.5))
model_mdata.add(Dense(32,activation='relu'))
model_mdata.add(Dense(num_classes,activation='softmax'))
model_mdata.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(5e-3),metrics=['accuracy'])
model_mdata.summary()

In [ ]:
import time
from tensorflow.keras.callbacks import ModelCheckpoint

st = time.time()

#create callback
filepath = 'my_best_model_LSTM_meta_data.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_accuracy',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')
callbacks = [checkpoint]


history=model_mdata.fit(x_train, y_train, validation_data=(x_dev, y_dev),epochs=5, batch_size=128, verbose=1,callbacks=callbacks)

elapsed_time = time.time() - st
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

# Bidirectionnel LSTM

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Bidirectional, LSTM, Embedding

In [ ]:
model_bidi_mdata=Sequential()
model_bidi_mdata.add(Embedding(len(list(unique_words)),150,input_length=183))
model_bidi_mdata.add(Bidirectional(LSTM(64,dropout=0.5, recurrent_dropout=0.5,return_sequences=True)))
model_bidi_mdata.add(LSTM(128,dropout=0.5, recurrent_dropout=0.5,return_sequences=True))
model_bidi_mdata.add(Bidirectional(LSTM(64,dropout=0.5, recurrent_dropout=0.5,return_sequences=True)))
model_bidi_mdata.add(Bidirectional(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False)))
model_bidi_mdata.add(Dense(64,activation='relu'))
model_bidi_mdata.add(Dropout(0.5))
model_bidi_mdata.add(Dense(32,activation='relu'))
model_bidi_mdata.add(Dropout(0.2))
model_bidi_mdata.add(Dense(num_classes,activation='softmax'))
model_bidi_mdata.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(5e-3),metrics=['accuracy'])
model_bidi_mdata.summary()

In [ ]:
import time
from tensorflow.keras.callbacks import ModelCheckpoint

st = time.time()

#create callback
filepath = 'my_best_model_bidirectionnel_meta_data.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_accuracy',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')
callbacks = [checkpoint]


history=model_bidi_mdata.fit(x_train, y_train, validation_data=(x_dev, y_dev),epochs=5, batch_size=128, verbose=1,callbacks=callbacks)

elapsed_time = time.time() - st
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))